<a href="https://colab.research.google.com/github/annieadhikary/ASD/blob/main/ASDClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import numpy as np
import tensorflow as tf
import pandas as pd
np.set_printoptions(precision=3, suppress=True)
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split

train = pd.read_csv('/home/dataset/Phenotypic_V1_0b_preprocessed1.csv')

train.head()

features = train.copy()
labels = features.pop('DX_GROUP')

# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Perform a calculation using the input
result = 2*input + 1

# the result doesn't have a value
result

calc = tf.keras.Model(inputs=input, outputs=result)

print(calc(1).numpy())
print(calc(2).numpy())

inputs = {}

for name, column in features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(train[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

preprocessed_inputs = [all_numeric_inputs]

print(inputs.items())

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  # Convert the feature values to string type
  feature_values = features[name].astype(str)

  lookup = layers.StringLookup(vocabulary=np.unique(feature_values))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

preprocessed_inputs

preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

tf.keras.utils.plot_model(model = preprocessing , rankdir="LR", dpi=72, show_shapes=True)

train_features_dict = {name: np.array(value) 
                         for name, value in features.items()}

features_dict = {name: values[:1].astype(str) if dtype == object else values[:1] for name, values in train_features_dict.items() for dtype in [values.dtype]}

preprocessing(features_dict)

def model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
  return model

model = model(preprocessing, inputs)

model.fit(x=features_dict, y=labels, epochs=10)



3.0
5.0
dict_items([('Unnamed: 0', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Unnamed: 0')>), ('Unnamed: 0.1', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Unnamed: 0.1')>), ('SUB_ID', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'SUB_ID')>), ('X', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'X')>), ('subject', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'subject')>), ('SITE_ID', <KerasTensor: shape=(None, 1) dtype=string (created by layer 'SITE_ID')>), ('FILE_ID', <KerasTensor: shape=(None, 1) dtype=string (created by layer 'FILE_ID')>), ('DSM_IV_TR', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'DSM_IV_TR')>), ('AGE_AT_SCAN', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'AGE_AT_SCAN')>), ('SEX', <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'SEX')>), ('HANDEDNESS_CATEGORY', <KerasTensor: shape=(None, 1) dtype=string (created by layer 'HANDEDNESS

ValueError: ignored